In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")

In [4]:
def question_decomposer_prompt(main_question: str) -> str:
    return f"""
    You are an expert researcher. Break down the following research question
    into 3-5 smaller sub-questions for detailed analysis:
    Question: "{main_question}"
    Return only the list of sub-questions.
    """

In [5]:
def summarization_prompt(sub_question: str, context: str) -> str:
    return f"""
    Summarize the following information to answer this sub-question:
    Sub-question: {sub_question}
    Context: {context}
    Answer in 3-4 concise sentences.
    """

In [6]:
def synthesis_prompt(main_question: str, sub_answers: str) -> str:
    return f"""
    You are an AI research assistant. Using the answers to sub-questions below,
    synthesize a detailed, coherent, and well-structured final report
    that addresses the main question:
    Main Question: {main_question}
    Sub-answers: {sub_answers}
    """

In [7]:
import requests

def search_web(query: str) -> str:
    # A simple Wikipedia search as a placeholder
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{query.replace(' ', '_')}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("extract", "No information found.")
    return "No information found."

In [8]:
# summarizer.py
from openai import OpenAI
from prompts import summarization_prompt

client = OpenAI(api_key=openai_api_key)

def summarize_text(sub_question: str, context: str) -> str:
    """
    Summarizes the context to answer the sub-question concisely.
    """
    prompt = summarization_prompt(sub_question, context)
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert summarizer."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()


In [9]:
from openai import OpenAI
from prompts import question_decomposer_prompt, synthesis_prompt
from retriever import search_web
from summarizer import summarize_text

client = OpenAI(api_key=openai_api_key)

    

In [10]:
def call_llm(prompt: str) -> str:
    """
    Call the LLM with a given prompt.
    """
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.4
    )
    return response.choices[0].message.content.strip()

In [11]:
def run_chain(main_question: str) -> str:
    # Step 1: Decompose the main question
    sub_questions = call_llm(question_decomposer_prompt(main_question)).split("\n")
    sub_answers = []

    # Step 2: For each sub-question, retrieve info and summarize
    for sub_q in sub_questions:
        sub_q = sub_q.strip("- ").strip()
        if sub_q:
            context = search_web(sub_q)
            answer = summarize_text(sub_q, context)
            sub_answers.append(f"{sub_q}: {answer}")

    # Step 3: Synthesize final report
    final_report = call_llm(synthesis_prompt(main_question, "\n".join(sub_answers)))
    return final_report

In [13]:
import gradio as gr

def chat_interface(question, history=[]):
    report = run_chain(question)
    return report


# Launch Gradio Chat Interface
print(gr.__version__)
gr.ChatInterface(
    fn=chat_interface,
    title="🤖 AI Research Assistant",
    description="Ask me any research question and I'll generate a detailed, well-structured report.",
    textbox=gr.Textbox(placeholder="Type your research question here...", container=True),
    theme="default",
    examples=["What are the impacts of AI on healthcare?"],
    
).launch()


5.38.2


/Volumes/Data/AI Projects/prompt-chaining-ai-research-assistant/.venv/lib/python3.13/site-packages/gradio/chat_interface.py:345: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
